In [1]:
import json
import requests
clientID = '0asbh4mg6ocktnlrvkjts9l29g9h8n'
secret = 'rifuh05a1wpxcwhrx75fqb8ga3aez3'
r = requests.post('https://id.twitch.tv/oauth2/token?client_id='+clientID+'&client_secret='+secret+'&grant_type=client_credentials')
token = json.loads(r.content)['access_token']

In [2]:
def get_id(user):
  headers = {'Client-ID': clientID, 'Accept': 'application/vnd.twitchtv.v5+json'}
  r = requests.get('https://api.twitch.tv/kraken/users?login='+user, headers =headers)
  obj = json.loads(r.content)
  try:
    return json.loads(r.content)['users'][0]['_id']
  except:
    return None

In [3]:
def get_user(id):
  headers = {'Client-ID': clientID, 'Accept': 'application/vnd.twitchtv.v5+json'}
  r = requests.get('https://api.twitch.tv/kraken/users/'+id, headers =headers)
  return json.loads(r.content)['display_name']

In [4]:
def follows_who(id):
  headers = {'Authorization': 'Bearer '+token, 'Client-ID': clientID, 'Accept': 'application/vnd.twitchtv.v5+json'}
  r = requests.get('https://api.twitch.tv/helix/users/follows?from_id='+id+'&first=100', headers =headers)
  res = json.loads(r.content)
  users = [name['to_name'] for name in res['data']]
  page = res['pagination'].get('cursor', None)
  while page:
    headers = {'Authorization': 'Bearer '+token, 'Client-ID': clientID, 'Accept': 'application/vnd.twitchtv.v5+json'}
    r = requests.get('https://api.twitch.tv/helix/users/follows?from_id='+id+'&first=100&after='+page, headers =headers)
    res = json.loads(r.content)
    new_users = [name['to_name'] for name in res['data']]
    users= users + new_users
    page = res['pagination'].get('cursor', None)
  return users

In [5]:
def compare(streamer1, streamer2):
  follow1 = follows_who(get_id(streamer1))
  follow2 = follows_who(get_id(streamer2))
  sim = [x for x in follow1 if x in follow2]
  print(sim)
  print(streamer1 +" and " + streamer2 + ' have ' + str(len(sim))
  + " followed streamers in common.")

In [63]:
compare('Destiny', 'loltyler1')

['MOONMOON', 'Macaiyla', 'xQcOW', 'AndyMilonakis', 'Greekgodx', 'Trainwreckstv', 'Reckful', 'Scarra', 'MitchJones']
Destiny and loltyler1 have 9 followed streamers in common.


In [53]:
from queue import Queue
def crawl(streamers):
  queue = Queue()
  for streamer in streamers:
    queue.put(streamer)
  visited = []
  edges = []
  while not queue.empty():
    name = queue.get()
    id = get_id(name)

    if id and name not in visited:
      visited.append(name)
      follows = follows_who(id)
      for person in follows:
        edges.append((name, person))
  return edges

In [74]:
edges = crawl(['BadBunny', 'Central_Committee', 'VaushVidya'])

In [75]:
len(edges)

659

In [79]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.DiGraph()


In [80]:
G.add_edges_from(edges)

In [81]:
nx.write_graphml(G, '/content/drive/MyDrive/Colab Notebooks/twitch_graph.graphml')